In [1]:
# ваше решение
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score


df = pd.read_csv('smoke_detector_task.csv')
df = df.drop(columns = ['Unnamed: 0'])
df

,UTC,Temperature[C],Humidity[%],TVOC[ppb],eCO2[ppm],Raw H2,Raw Ethanol,Pressure[hPa],PM1.0,PM2.5,NC0.5,NC1.0,NC2.5,CNT,Fire Alarm
0,1654733331,20.000,57.36,0.0,400.0,12306.0,18520,939.735,0.00,0.00,0.00,0.000,0.000,0,No
1,1654733332,20.015,56.67,0.0,400.0,12345.0,18651,939.744,0.00,0.00,0.00,0.000,0.000,1,No
2,1654733333,20.029,55.96,0.0,400.0,12374.0,18764,939.738,0.00,0.00,0.00,0.000,0.000,2,No
3,1654733334,20.044,55.28,0.0,400.0,12390.0,18849,939.736,0.00,0.00,0.00,0.000,0.000,3,No
4,1654733335,20.059,54.69,0.0,400.0,12403.0,18921,939.744,0.00,0.00,0.00,0.000,0.000,4,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62625,1655130047,18.438,15.79,625.0,400.0,13723.0,20569,936.670,0.63,0.65,4.32,0.673,0.015,5739,No
62626,1655130048,18.653,15.87,612.0,400.0,13731.0,20588,936.678,0.61,0.63,4.18,0.652,0.015,5740,No
62627,1655130049,18.867,15.84,627.0,400.0,13725.0,20582,936.687,0.57,0.60,3.95,0.617,0.014,5741,No
62628,1655130050,19.083,16.04,638.0,400.0,13712.0,20566,936.680,0.57,0.59,3.92,0.611,0.014,5742,No


In [2]:
columns_to_repair = ['Temperature[C]', 'Humidity[%]', 'TVOC[ppb]', 'eCO2[ppm]', 'Raw H2', 'Pressure[hPa]', 'PM2.5', 'NC1.0']
for i in columns_to_repair:
    df.loc[df[i].isna(), i] = df[i].mean()


In [3]:
X = df.drop(columns = ['Fire Alarm'])
y = df['Fire Alarm']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [21]:
from sklearn import svm
from sklearn.metrics import classification_report

clf = svm.SVC()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
y_pred = pd.Series(y_pred)
print(classification_report(pd.Series(list(y_test)), y_pred))

D:\ml\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

          No       0.00      0.00      0.00      5324
         Yes       0.72      1.00      0.83     13465

    accuracy                           0.72     18789
   macro avg       0.36      0.50      0.42     18789
weighted avg       0.51      0.72      0.60     18789



D:\ml\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\ml\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:
y_pred

0        Yes
1        Yes
2        Yes
3        Yes
4        Yes
        ... 
18784    Yes
18785    Yes
18786    Yes
18787    Yes
18788    Yes
Length: 18789, dtype: object

In [23]:
pd.Series(list(y_test))

0         No
1         No
2        Yes
3        Yes
4        Yes
        ... 
18784    Yes
18785     No
18786     No
18787     No
18788    Yes
Length: 18789, dtype: object

In [4]:
# https://scikit-learn.org/stable/modules/naive_bayes.html
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report


gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          No       0.88      0.45      0.59      5372
         Yes       0.81      0.98      0.89     13417

    accuracy                           0.82     18789
   macro avg       0.85      0.71      0.74     18789
weighted avg       0.83      0.82      0.80     18789



In [13]:
pd.DataFrame({'1':y_pred}).value_counts()

1  
Yes    15960
No      2829
dtype: int64

In [54]:
class NB:
    def __init__(self):
        self.classes = None
        self.M = None
        self.STD = None
        
    def fit(self, X, y):
        self.classes = np.unique(y)
        print(self.classes)
        self.M = np.zeros((len(self.classes), X.shape[1]))
        for i in range(len(self.classes)):
            self.M[i] = np.mean(X[y==self.classes[i]], axis = 0)

        self.STD = np.zeros((len(self.classes), X.shape[1]))
        for i in range(len(self.classes)):
            self.STD[i] = np.std(X[y==self.classes[i]], axis = 0)
    
    def predict(self, X):
        prob = [0] * len(self.classes)
        for i in range(len(prob)):
            prob[i] = list((np.sum((-1)*(X - self.M[i])/self.STD[i], axis = 1)))
            
        return np.argmax(prob, axis = 0)
    
        

In [59]:
Test = NB()
Test.fit(X_train, y_train)
y_pred = list(Test.predict(X_test))
for i in range(len(y_pred)):
    if y_pred[i] == 0:
        y_pred[i] = 'No'
    if y_pred[i] == 1:
        y_pred[i] = 'Yes'

['No' 'Yes']
